In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import sqlite3
from datetime import datetime


In [17]:
def get_shadow_root(driver):
    shadow_host = driver.find_element(By.CSS_SELECTOR, '#usercentrics-root')
    return shadow_host.shadow_root

def click_shadow_button(shadow_root):
    shadow_container = shadow_root.find_element(By.CSS_SELECTOR, '#uc-center-container')
    shadow_btns = shadow_container.find_elements(By.TAG_NAME, 'button')
    shadow_btns[2].click()

def scroll_to_bottom(driver):
    driver.execute_script("window.scrollTo(0, 1000);")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

def get_product_data(driver):
    product_names = driver.find_elements(By.XPATH, '//a[@data-test-id="product-card__productName"]')
    product_prices = driver.find_elements(By.XPATH, '//div[@data-test-id="product-card__productPrice__comparisonPrice"]')
    return list(zip(product_names, product_prices))
    
def save_to_database(data):
    conn = sqlite3.connect("suomi_milk_product_data.db")
    c = conn.cursor()
    c.execute("CREATE TABLE IF NOT EXISTS products (id INTEGER PRIMARY KEY, product_name TEXT, product_price TEXT, date TEXT)")
    today = datetime.now().strftime("%Y-%m-%d")
    for index, (product_name, product_price) in enumerate(data):
        c.execute("INSERT INTO products (id, product_name, product_price, date) VALUES (?, ?, ?, ?)", (index, product_name.text, product_price.text, today))
    conn.commit()
    conn.close()
    
def collect_data():
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    url = 'https://www.s-kaupat.fi/tuotteet/maito-munat-ja-rasvat-0/maidot-ja-piimat/maidot?queryString=maito'
    driver.get(url)
    driver.implicitly_wait(3)
    scroll_to_bottom(driver)
    shadow_root = get_shadow_root(driver)
    click_shadow_button(shadow_root)
    scroll_to_bottom(driver)
    time.sleep(3)
    product_data = get_product_data(driver)
    for index, (product_name, product_price) in enumerate(product_data):
        print(index, product_name.text)
        print(product_price.text)
    save_to_database(product_data)
    driver.close()

if __name__ == '__main__':
    collect_data()

0 Kotimaista rasvaton maito 1l
0,85 €/l
1 Valio Hyvä suomalainen Arki® kevytmaitojuoma 1 l
0,89 €/l
2 Kotimaista täysmaito 1l
1,19 €/l
3 Valio Hyvä suomalainen Arki® rasvaton maitojuoma 1 l
0,79 €/l
4 Valio vapaan lehmän kevytmaito 1 l
1,19 €/l
5 Juustoportti Vapaan Lehmän rasvaton maito 1 l
1,49 €/l
6 Valio vapaan lehmän täysmaito 1 l
1,39 €/l
7 Valio kevytmaito 5 dl
1,98 €/l
8 Juustoportti Vapaan lehmän kevytmaito 1 l
1,65 €/l
9 Juustoportti Vapaan lehmän täysmaito 1 l
1,79 €/l
10 Arla 1,5 L rasvaton maito Suomesta
0,97 €/l
11 Valio Hyvä suomalainen Arki® rasvaton maitojuoma 1,75 l
0,77 €/l
12 Valio Luomu™ rasvaton maito 1 l
1,19 €/l
13 Valio Hyvä suomalainen Arki® kevytmaitojuoma 1,75 l
0,87 €/l
14 Arla 1 L laktoositon Cafè UHT maito
2,09 €/l
15 Valio Luomu™ D-vitaminoitu rasvaton maito 1 l
1,19 €/l
16 Arla 2 dl Suomi ESL kevyt maito
2,45 €/l
17 Valio Luomu™ rasvaton maito 1,75 l
1,14 €/l
18 Valio Luomu™ kevytmaito 1 l
1,25 €/l
19 Arla 1,5 L Suomesta kevytmaito
1,06 €/l
20 Kotimaist

In [18]:
def read_data():
    conn = sqlite3.connect('suomi_milk_product_data.db')
    c = conn.cursor()
    c.execute("SELECT * FROM products")
    rows = c.fetchall()
    for row in rows:
        print(row)
    conn.close()

if __name__ == '__main__':
    read_data()

(0, 'Kotimaista rasvaton maito 1l', '0,85 €/l', '2023-01-30')
(1, 'Valio Hyvä suomalainen Arki® kevytmaitojuoma 1 l', '0,89 €/l', '2023-01-30')
(2, 'Kotimaista täysmaito 1l', '1,19 €/l', '2023-01-30')
(3, 'Valio Hyvä suomalainen Arki® rasvaton maitojuoma 1 l', '0,79 €/l', '2023-01-30')
(4, 'Valio vapaan lehmän kevytmaito 1 l', '1,19 €/l', '2023-01-30')
(5, 'Juustoportti Vapaan Lehmän rasvaton maito 1 l', '1,49 €/l', '2023-01-30')
(6, 'Valio vapaan lehmän täysmaito 1 l', '1,39 €/l', '2023-01-30')
(7, 'Valio kevytmaito 5 dl', '1,98 €/l', '2023-01-30')
(8, 'Juustoportti Vapaan lehmän kevytmaito 1 l', '1,65 €/l', '2023-01-30')
(9, 'Juustoportti Vapaan lehmän täysmaito 1 l', '1,79 €/l', '2023-01-30')
(10, 'Arla 1,5 L rasvaton maito Suomesta', '0,97 €/l', '2023-01-30')
(11, 'Valio Hyvä suomalainen Arki® rasvaton maitojuoma 1,75 l', '0,77 €/l', '2023-01-30')
(12, 'Valio Luomu™ rasvaton maito 1 l', '1,19 €/l', '2023-01-30')
(13, 'Valio Hyvä suomalainen Arki® kevytmaitojuoma 1,75 l', '0,87 €/l'